In [ ]:
import pickle
from graph import Graph, Part
from typing import List, Callable, Set
from sklearn.model_selection import train_test_split

with open('data/graphs.dat', 'rb') as file:
    all_graphs: List[Graph] = pickle.load(file)
    X_train, X_temp, y_train, y_temp = train_test_split(list(map(lambda g: g.get_parts(), all_graphs)), all_graphs, test_size=0.3, random_state=0)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [ ]:
from node import Node

class Ordering:
    def __init__(self, y: List[Graph], degreeAggregate: Callable[[List[int]], int]):
        degrees = {}
        for graph in y:
            for (node, edges) in graph.get_edges().items():
                if node.get_part().get_part_id() not in degrees:
                    degrees[node.get_part().get_part_id()] = []
                degrees[node.get_part().get_part_id()].append(len(edges))
        self.keys = {part: degreeAggregate(degs) for (part, degs) in degrees.items()}

    def sort(self, x: Set[Part]) -> List[Part]:
        return sorted(x, key=lambda n: self.keys[n.get_part_id()])
    
    def test_graphs(self, graphs: List[Graph]) -> float:
        problems: int = 0
        for graph in graphs:
            edges = {node: edges.copy() for (node, edges) in graph.get_edges().items()}
            nodes: List[Node] = sorted(graph.get_nodes(), key=lambda n: self.keys[n.get_part().get_part_id()])
            for node in nodes[:-1]:
                if len(edges[node]) != 1:
                    problems += 1
                    break
                parent = edges[node][0]
                edges[parent].remove(node)
        return 1 - (problems / len(graphs))

avgOrder = Ordering(y_train, lambda n: sum(n)/len(n))
print(avgOrder.test_graphs(y_val))